# Initialized

In [1]:
# At end of session
import duckdb as ddb
ddb.close()

In [75]:
import os
from dotenv import load_dotenv
import pandas as pd
import duckdb as ddb
import requests
import datetime
import time
import pandas_ta as ta

from SQL.Querys import DBconn
con = ddb.connect(r'C:\Design Folder\RBGithub\Trading_Algo_Data\testingalphaoptions.duckdb')
db = DBconn(con)

date = '2024-07-05'

### Get the Stock Data form Alpha Vantage

In [102]:
load_dotenv()
api_key = os.getenv("Alpha_API")
symbol = 'AAPL'
interval = '1'
Stock_lookback_period = 100
Increasing_Mins = 2

#url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=1min&month=2024-07&outputsize=full&apikey={api_key}"

stock_api = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}min&entitlement=realtime&outputsize=full&apikey={api_key}'


r = requests.get(stock_api)
key = f'Time Series (1min)'
data = r.json().get(key, {})

df = pd.DataFrame.from_dict(data, orient='index')

df.index = pd.to_datetime(df.index)

stock_df = df.loc[date]

new_column_names = ['open', 'high', 'low', 'close', 'volume']
stock_df.columns = new_column_names


stock_df = stock_df.copy()
stock_df['timestamp'] = stock_df.index
stock_df.reset_index(drop=True,inplace=True)

stock_df['Min_Test'] = False
stock_df['EmaMin_Test'] = False
stock_df['EMA_Test'] = False
stock_df['Trade'] = False


print(stock_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960 entries, 0 to 959
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   open         960 non-null    object        
 1   high         960 non-null    object        
 2   low          960 non-null    object        
 3   close        960 non-null    object        
 4   volume       960 non-null    object        
 5   timestamp    960 non-null    datetime64[ns]
 6   Min_Test     960 non-null    bool          
 7   EmaMin_Test  960 non-null    bool          
 8   EMA_Test     960 non-null    bool          
 9   Trade        960 non-null    bool          
dtypes: bool(4), datetime64[ns](1), object(5)
memory usage: 48.9+ KB
None


# Analyze Stock Data
1. Stock_df index Starts with newest stock price
2. Swoosh
   - Price should be greater than 4
   - 1 should be greater than 3
   - 3 should be greater than 2

### Notes
1. Swoosh Points tell us When to look at the data 
2. 94 / 959 are Swoosh Points
3. Analysing Local Minmums
   1. Find all the minimums
   2. Check to see if the last X minimums are increases
   3. Check if the last X minimums are > 8 EMA
   4. Check if for the last X Minimums the 8 EMA > 20 EMA

In [105]:

stock_df['close'] = pd.to_numeric(stock_df['close'])

stock_df['less1'] = stock_df['close'].shift(-1)
stock_df['less2'] = stock_df['close'].shift(-2)
stock_df['less3'] = stock_df['close'].shift(-3)
stock_df['Swoosh'] = (stock_df['less1']<stock_df['close']) & (stock_df['less2']<stock_df['less1']) & (stock_df['less2']<stock_df['less3'])

swoosh_points = stock_df[stock_df['Swoosh']]

for index, row in swoosh_points.iterrows():
    if index >= Stock_lookback_period: 
        table_range = stock_df.iloc[index-Stock_lookback_period:index].copy()

        # Stock Analysis 
        # Reverse table to calculate EMA correctly
        table_range_reversed = table_range.iloc[::-1].copy()
        table_range_reversed['EMA8'] = ta.ema(close=table_range_reversed['close'], length=8)
        table_range_reversed['EMA20'] = ta.ema(close=table_range_reversed['close'], length=20)

        # Reverse the EMA columns back to original order
        table_range['EMA8'] = table_range_reversed['EMA8'].iloc[::-1].values
        table_range['EMA20'] = table_range_reversed['EMA20'].iloc[::-1].values

        table_range['Local_Min'] = table_range['close'][(table_range['close'].shift(-1)>table_range['close'])&
                                                        (table_range['close'].shift(1)>table_range['close'])]

        local_mins = table_range[['close','Local_Min','EMA8','EMA20']].dropna()

        if len(local_mins) < 2:
            continue

        local_mins['Min_Test'] = local_mins['Local_Min'] > local_mins['Local_Min'].shift(-1)
        local_mins['EmaMin_Test'] = local_mins['Local_Min'] > local_mins['EMA8']
        local_mins['EMA_Test'] = local_mins['EMA8'] > local_mins['EMA20']
        
        Update_df = local_mins[['Min_Test','EmaMin_Test','EMA_Test']]

        stock_df.update(Update_df)

        if all([local_mins['Min_Test'].iloc[0:2].all(),
                local_mins['EmaMin_Test'].iloc[0:1].all(),
                local_mins['EMA_Test'].iloc[0:1].all()]):
            stock_df.at[index, 'Trade'] = True

        
        with pd.ExcelWriter('stockdata3.xlsx') as writer:
            stock_df.to_excel(writer, sheet_name='Sheet1', index=True)


print(stock_df[stock_df['Trade']].info())





<class 'pandas.core.frame.DataFrame'>
Index: 28 entries, 284 to 953
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   open         28 non-null     object        
 1   high         28 non-null     object        
 2   low          28 non-null     object        
 3   close        28 non-null     float64       
 4   volume       28 non-null     object        
 5   timestamp    28 non-null     datetime64[ns]
 6   Min_Test     28 non-null     object        
 7   EmaMin_Test  28 non-null     object        
 8   EMA_Test     28 non-null     object        
 9   Trade        28 non-null     bool          
 10  less1        28 non-null     float64       
 11  less2        28 non-null     float64       
 12  less3        28 non-null     float64       
 13  Swoosh       28 non-null     bool          
dtypes: bool(2), datetime64[ns](1), float64(4), object(7)
memory usage: 2.9+ KB
None


### Getting Data From Options DB 
1. Get a List of Options Log Times
2. Write a For Loop that goes through 

In [ ]:
test_df = stock_df.copy()

table_range_reversed = test_df.iloc[::-1].copy()
table_range_reversed['EMA8'] = ta.ema(close=table_range_reversed['close'], length=8)
table_range_reversed['EMA20'] = ta.ema(close=table_range_reversed['close'], length=20)

# Reverse the EMA columns back to original order
test_df['EMA8'] = table_range_reversed['EMA8'].iloc[::-1].values
test_df['EMA20'] = table_range_reversed['EMA20'].iloc[::-1].values

test_df['Local_Min'] = test_df['close'][(test_df['close'].shift(-1)>test_df['close'])&
                                                (test_df['close'].shift(1)>test_df['close'])]


with pd.ExcelWriter('Stockdf2.xlsx') as writer:
  test_df.to_excel(writer, sheet_name='Sheet1', index=True)